In [1]:
# Importar las librerías necesarias
import pandas as pd
from tabulate import tabulate
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
# Cargar archivo XLS
# path = '../data/raw/Boletin_CUADRO_17_db.xlsx'
# df = pd.read_excel(path)
path = '../data/raw/data.csv'
# df = pd.read_excel(path, dtype_backend='pyarrow')
df = pd.read_csv(path)
print(df.shape)
df.head()

(329637, 14)


,Año,Semana,Pag.,Cuadro,No_01,Entidad,No_02,Padecimiento,Ax_001,Ax_002,Ax_003,No_03,Valor,No_04
0,2014,sem02,57,CUADRO 17.,NaN,Aguascalientes,NaN,DepresiónCIE-10ª REV.F32,2014,0,Sem.,NaN,1,NaN
1,2014,sem02,57,CUADRO 17.,NaN,Aguascalientes,NaN,DepresiónCIE-10ª REV.F32,2014,Acum.,H,NaN,1,NaN
2,2014,sem02,57,CUADRO 17.,NaN,Aguascalientes,NaN,DepresiónCIE-10ª REV.F32,2014,Acum.,M,NaN,-,NaN
3,2014,sem02,57,CUADRO 17.,NaN,Aguascalientes,NaN,Enfermedad de Parkinson\nCIE-10ª REV.\nG20,2014,0,Sem.,NaN,-,NaN
4,2014,sem02,57,CUADRO 17.,NaN,Aguascalientes,NaN,Enfermedad de Parkinson\nCIE-10ª REV.\nG20,2014,Acum.,H,NaN,-,NaN


In [3]:
new_columns = {'Cuadro ': 'Cuadro', 'Entidad': 'Estado'}
df.rename(columns=new_columns, inplace=True)

In [4]:
# Análisis exploratorio básico
print('Valores nulos por columna:')
print(tabulate(df.isnull().sum().reset_index(), 
               headers=['Columna', 'Valores nulos'], 
               tablefmt='rounded_outline', 
               showindex=False))

print('\nValores únicos por columna:')
print(tabulate(df.nunique().reset_index(), 
               headers=['Columna', 'Valores únicos'], 
               tablefmt='rounded_outline', 
               showindex=False))

print('\nTipos de datos por columna:')
print(tabulate(df.dtypes.reset_index(), 
               headers=['Columna', 'Tipo de dato'], 
               tablefmt='rounded_outline', 
               showindex=False))

Valores nulos por columna:
╭──────────────┬─────────────────╮
│ Columna      │   Valores nulos │
├──────────────┼─────────────────┤
│ Año          │               0 │
│ Semana       │               0 │
│ Pag.         │               0 │
│ Cuadro       │               0 │
│ No_01        │          329637 │
│ Estado       │               0 │
│ No_02        │          329637 │
│ Padecimiento │               0 │
│ Ax_001       │               0 │
│ Ax_002       │               0 │
│ Ax_003       │               0 │
│ No_03        │          329637 │
│ Valor        │               0 │
│ No_04        │          329637 │
╰──────────────┴─────────────────╯

Valores únicos por columna:
╭──────────────┬──────────────────╮
│ Columna      │   Valores únicos │
├──────────────┼──────────────────┤
│ Año          │               11 │
│ Semana       │               53 │
│ Pag.         │               26 │
│ Cuadro       │                4 │
│ No_01        │                0 │
│ Estado       │          

In [5]:
df['Semana'] = df['Semana'].str.replace('sem', '').astype('int')
df['Fecha'] = pd.to_datetime(df['Año'].astype(str) + df['Semana'].astype(str) + '1', format='%G%V%u')
df['Valor'] = pd.to_numeric(df['Valor'], errors='coerce').fillna(0).astype(int)
df['Padecimiento'] = df['Padecimiento'].astype(str).str.replace('\n', '')
df['Codigo Padecimiento'] = df['Padecimiento'].str.extract(r'REV\.(.*)')
# Eliminar las columnas que tengan el prefijo "No" (Columnas Vacías)
df = df.loc[:, ~df.columns.str.startswith('No')]
# Eliminar la columna Pag.
df.drop(columns='Pag.', inplace=True)
df.head(5)

,Año,Semana,Cuadro,Estado,Padecimiento,Ax_001,Ax_002,Ax_003,Valor,Fecha,Codigo Padecimiento
0,2014,2,CUADRO 17.,Aguascalientes,DepresiónCIE-10ª REV.F32,2014,0,Sem.,1,2014-01-06,F32
1,2014,2,CUADRO 17.,Aguascalientes,DepresiónCIE-10ª REV.F32,2014,Acum.,H,1,2014-01-06,F32
2,2014,2,CUADRO 17.,Aguascalientes,DepresiónCIE-10ª REV.F32,2014,Acum.,M,0,2014-01-06,F32
3,2014,2,CUADRO 17.,Aguascalientes,Enfermedad de ParkinsonCIE-10ª REV.G20,2014,0,Sem.,0,2014-01-06,G20
4,2014,2,CUADRO 17.,Aguascalientes,Enfermedad de ParkinsonCIE-10ª REV.G20,2014,Acum.,H,0,2014-01-06,G20


In [6]:
columnas_grupo  = ['Cuadro', 'Estado', 'Codigo Padecimiento', 'Ax_003']
valores_unicos = {}
for col in columnas_grupo:
    valores_unicos[col] = df[col].unique()
display(valores_unicos)
# cuantas combinaciones unicas hay
combinaciones_unicas = df[columnas_grupo].drop_duplicates()
print(combinaciones_unicas.shape)
print(tabulate(combinaciones_unicas.head(10), headers='keys', tablefmt='rounded_outline'))

# Se agrega una columna nueva llamada 'key' concatenando las columnas de agrupacion
# df['key'] = df[columnas_grupo].astype(str).agg('|'.join, axis=1)
# print(df['key'].nunique())

{'Cuadro': array(['CUADRO 17.', 'CUADRO 17.1', 'CUADRO 17.2', 'CUADRO 17.3'],
       dtype=object),
 'Estado': array(['Aguascalientes', 'Baja California', 'Baja California Sur',
        'Campeche', 'Coahuila', 'Colima', 'Chiapas', 'Chihuahua',
        'Distrito Federal', 'Durango', 'Guanajuato', 'Guerrero', 'Hidalgo',
        'Jalisco', 'México', 'Michoacán', 'Morelos', 'Nayarit',
        'Nuevo León', 'Oaxaca', 'Puebla', 'Querétaro', 'Quintana Roo',
        'San Luis Potosí', 'Sinaloa', 'Sonora', 'Tabasco', 'Tamaulipas',
        'Tlaxcala', 'Veracruz', 'Yucatán', 'Zacatecas', 'TOTAL',
        'Ciudad de México'], dtype=object),
 'Codigo Padecimiento': array(['F32', 'G20', 'G30', 'X60', 'X61', 'X70', 'X72', 'X78', 'X80',
        'Z91.5', 'R45.8'], dtype=object),
 'Ax_003': array(['Sem.', 'H', 'M', 'Acum.'], dtype=object)}

(1464, 4)
╭────┬────────────┬─────────────────┬───────────────────────┬──────────╮
│    │ Cuadro     │ Estado          │ Codigo Padecimiento   │ Ax_003   │
├────┼────────────┼─────────────────┼───────────────────────┼──────────┤
│  0 │ CUADRO 17. │ Aguascalientes  │ F32                   │ Sem.     │
│  1 │ CUADRO 17. │ Aguascalientes  │ F32                   │ H        │
│  2 │ CUADRO 17. │ Aguascalientes  │ F32                   │ M        │
│  3 │ CUADRO 17. │ Aguascalientes  │ G20                   │ Sem.     │
│  4 │ CUADRO 17. │ Aguascalientes  │ G20                   │ H        │
│  5 │ CUADRO 17. │ Aguascalientes  │ G20                   │ M        │
│  6 │ CUADRO 17. │ Aguascalientes  │ G30                   │ Sem.     │
│  7 │ CUADRO 17. │ Aguascalientes  │ G30                   │ H        │
│  8 │ CUADRO 17. │ Aguascalientes  │ G30                   │ M        │
│  9 │ CUADRO 17. │ Baja California │ F32                   │ Sem.     │
╰────┴────────────┴─────────────────┴────

In [7]:
for año in df['Año'].unique():
    un = df[df['Año']==año]['Ax_001'].unique()
    print(f'Año: {año}\nValores Unicos: {un}' )

Año: 2014
Valores Unicos: [2014    0]
Año: 2015
Valores Unicos: [2014 2015]
Año: 2016
Valores Unicos: [2015 2014 2016]
Año: 2017
Valores Unicos: [2016 2015 2017]
Año: 2018
Valores Unicos: [2017 2016 2018]
Año: 2019
Valores Unicos: [2018 2017 2019]
Año: 2020
Valores Unicos: [2019 2018 2020]
Año: 2021
Valores Unicos: [2020 2019 2021]
Año: 2022
Valores Unicos: [2021 2020 2022]
Año: 2023
Valores Unicos: [2022 2021 2023    0]
Año: 2024
Valores Unicos: [2023 2022 2024    0]


In [8]:
faltantes_dict = {}
fechas_completas = pd.date_range(df["Fecha"].min(), df["Fecha"].max(), freq="W-MON")
# Agrupar por las combinaciones que te interesan
for keys, grupo in df.groupby(columnas_grupo):
    fechas_existentes = grupo["Fecha"].unique()
    # fechas_completas = pd.date_range(
    #     grupo["Fecha"].min(), grupo["Fecha"].max(), freq="W-MON"
    # )
    faltantes = fechas_completas.difference(fechas_existentes)
    if len(faltantes) > 0:
        faltantes_dict[keys] = faltantes

# Mostrar resultados
for combo, f in faltantes_dict.items():
    print(f"\n🔸 Combinación {combo} tiene {len(f)} fechas faltantes:")
    print(f)


🔸 Combinación ('CUADRO 17.', 'Aguascalientes', 'F32', 'Acum.') tiene 54 fechas faltantes:
DatetimeIndex(['2014-01-06', '2014-01-13', '2014-01-20', '2014-01-27',
               '2014-02-03', '2014-02-10', '2014-02-17', '2014-02-24',
               '2014-03-03', '2014-03-10', '2014-03-17', '2014-03-24',
               '2014-03-31', '2014-04-07', '2014-04-14', '2014-04-21',
               '2014-04-28', '2014-05-05', '2014-05-12', '2014-05-19',
               '2014-05-26', '2014-06-02', '2014-06-09', '2014-06-16',
               '2014-06-23', '2014-06-30', '2014-07-07', '2014-07-14',
               '2014-07-21', '2014-07-28', '2014-08-04', '2014-08-11',
               '2014-08-18', '2014-08-25', '2014-09-01', '2014-09-08',
               '2014-09-15', '2014-09-22', '2014-09-29', '2014-10-06',
               '2014-10-13', '2014-10-20', '2014-10-27', '2014-11-03',
               '2014-11-10', '2014-11-17', '2014-11-24', '2014-12-01',
               '2014-12-08', '2014-12-15', '2014-12-22', 

In [9]:
# Todas las combinaciones únicas
combos = df[columnas_grupo].drop_duplicates()

# Rango de fechas general (mínimo y máximo en todo el df)
fechas_completas = pd.date_range(df["Fecha"].min(), df["Fecha"].max(), freq="W-MON")

# Producto cartesiano (todas las combinaciones × todas las semanas)
idx = pd.MultiIndex.from_product(
    [combos.itertuples(index=False, name=None), fechas_completas],
    names=["Combo", "Fecha"]
)
df_completo = pd.DataFrame(index=idx).reset_index()
# display(df_completo.head(4))
# Expandir las columnas del combo
df_completo[columnas_grupo] = pd.DataFrame(
    df_completo["Combo"].tolist(), index=df_completo.index
)
# display(df_completo.head(4))
df_completo = df_completo.drop(columns="Combo")

# Unir con tu df original
df_merge = df_completo.merge(
    df, on=["Fecha", "Cuadro", "Estado", "Ax_003", "Codigo Padecimiento"], how="left"
)

# Filtrar los registros faltantes
faltantes = df_merge[df_merge.isna().any(axis=1)]

In [10]:
df_merge.head()

,Fecha,Cuadro,Estado,Codigo Padecimiento,Ax_003,Año,Semana,Padecimiento,Ax_001,Ax_002,Valor
0,2014-01-06,CUADRO 17.,Aguascalientes,F32,Sem.,2014.0,2.0,DepresiónCIE-10ª REV.F32,2014.0,0,1.0
1,2014-01-13,CUADRO 17.,Aguascalientes,F32,Sem.,2014.0,3.0,DepresiónCIE-10ª REV.F32,2014.0,0,2.0
2,2014-01-20,CUADRO 17.,Aguascalientes,F32,Sem.,2014.0,4.0,DepresiónCIE-10ª REV.F32,2014.0,0,1.0
3,2014-01-27,CUADRO 17.,Aguascalientes,F32,Sem.,2014.0,5.0,DepresiónCIE-10ª REV.F32,2014.0,0,1.0
4,2014-02-03,CUADRO 17.,Aguascalientes,F32,Sem.,2014.0,6.0,DepresiónCIE-10ª REV.F32,2014.0,0,2.0


In [11]:
df_ref = df[["Codigo Padecimiento", "Padecimiento"]].drop_duplicates()
df_final = df_merge.merge(df_ref, on="Codigo Padecimiento", how="right", suffixes=("_ref", ""))
# df_final = df_merge.copy()
df_final["Año"] = df_final["Fecha"].dt.isocalendar().year
df_final["Semana"] = df_final["Fecha"].dt.isocalendar().week
df_final

,Fecha,Cuadro,Estado,Codigo Padecimiento,Ax_003,Año,Semana,Padecimiento_ref,Ax_001,Ax_002,Valor,Padecimiento
0,2014-01-06,CUADRO 17.,Aguascalientes,F32,Sem.,2014,2,DepresiónCIE-10ª REV.F32,2014.0,0,1.0,DepresiónCIE-10ª REV.F32
1,2014-01-13,CUADRO 17.,Aguascalientes,F32,Sem.,2014,3,DepresiónCIE-10ª REV.F32,2014.0,0,2.0,DepresiónCIE-10ª REV.F32
2,2014-01-20,CUADRO 17.,Aguascalientes,F32,Sem.,2014,4,DepresiónCIE-10ª REV.F32,2014.0,0,1.0,DepresiónCIE-10ª REV.F32
3,2014-01-27,CUADRO 17.,Aguascalientes,F32,Sem.,2014,5,DepresiónCIE-10ª REV.F32,2014.0,0,1.0,DepresiónCIE-10ª REV.F32
4,2014-02-03,CUADRO 17.,Aguascalientes,F32,Sem.,2014,6,DepresiónCIE-10ª REV.F32,2014.0,0,2.0,DepresiónCIE-10ª REV.F32
...,...,...,...,...,...,...,...,...,...,...,...,...
839164,2024-11-25,CUADRO 17.3,TOTAL,R45.8,Acum.,2024,48,Otros Síntomas y Signos que Involucran el Esta...,2023.0,0,1799.0,Otros Síntomas y Signos que Involucran el Esta...
839165,2024-12-02,CUADRO 17.3,TOTAL,R45.8,Acum.,2024,49,Otros Síntomas y Signos que Involucran el Esta...,2023.0,0,1824.0,Otros Síntomas y Signos que Involucran el Esta...
839166,2024-12-09,CUADRO 17.3,TOTAL,R45.8,Acum.,2024,50,Otros Síntomas y Signos que Involucran el Esta...,2023.0,0,1859.0,Otros Síntomas y Signos que Involucran el Esta...
839167,2024-12-16,CUADRO 17.3,TOTAL,R45.8,Acum.,2024,51,Otros Síntomas y Signos que Involucran el Esta...,2023.0,0,1882.0,Otros Síntomas y Signos que Involucran el Esta...


In [12]:
print('Valores nulos por columna:')
print(tabulate(df_final.isnull().sum().reset_index(), 
               headers=['Columna', 'Valores nulos'], 
               tablefmt='rounded_outline', 
               showindex=False))

Valores nulos por columna:
╭─────────────────────┬─────────────────╮
│ Columna             │   Valores nulos │
├─────────────────────┼─────────────────┤
│ Fecha               │               0 │
│ Cuadro              │               0 │
│ Estado              │               0 │
│ Codigo Padecimiento │               0 │
│ Ax_003              │               0 │
│ Año                 │               0 │
│ Semana              │               0 │
│ Padecimiento_ref    │          509532 │
│ Ax_001              │          509532 │
│ Ax_002              │          509532 │
│ Valor               │          509532 │
│ Padecimiento        │               0 │
╰─────────────────────┴─────────────────╯


In [13]:
# # key = 'CUADRO 17.|Aguascalientes|G20|Acum.'
# key = 'CUADRO 17.|Michoacán|G20|Acum.'
filters = (df_final['Cuadro'] == 'CUADRO 17.') \
          & (df_final['Estado']== 'Chihuahua') \
          & (df_final['Codigo Padecimiento'] == 'F32') \
          & (df_final['Ax_003'] == 'Sem.') 
        #   & ((df['Año'] >= 2014) & (df['Año'] <= 2016))    
# filters = (df['key']==key)

tmp = df_final[filters]
display(tmp.isna().sum())
# display(tmp)
# Linea de tiempo con Seaborn
fig = px.line(tmp, x='Fecha', y='Valor',title=f"Serie de tiempo para: ", 
              hover_data={'Semana': True, 'Año': True},
              color = 'Ax_003', markers = True)
# Scatter plot con Plotly
# fig = px.scatter(tmp, x='Fecha', y='Valor', title=f"Dispersión para: ",
#                  hover_data={'Semana': True, 'Año': True},
#                  color = 'Ax_003')
fig.update_layout(height=800)
fig.show()

Fecha                  0
Cuadro                 0
Estado                 0
Codigo Padecimiento    0
Ax_003                 0
Año                    0
Semana                 0
Padecimiento_ref       2
Ax_001                 2
Ax_002                 2
Valor                  2
Padecimiento           0
dtype: int64

In [14]:
df_final.dtypes

Fecha                  datetime64[ns]
Cuadro                         object
Estado                         object
Codigo Padecimiento            object
Ax_003                         object
Año                            UInt32
Semana                         UInt32
Padecimiento_ref               object
Ax_001                        float64
Ax_002                         object
Valor                         float64
Padecimiento                   object
dtype: object

In [15]:
# # key = 'CUADRO 17.|Aguascalientes|G20|Acum.'
# key = 'CUADRO 17.|Michoacán|G20|Acum.'
filters = (df_final['Cuadro'] == 'CUADRO 17.') \
          & (df_final['Estado']== 'Chihuahua') \
          & (df_final['Codigo Padecimiento'] == 'F32') \
          & (df_final['Ax_003'] == 'Acum.') 
        #   & ((df['Año'] >= 2014) & (df['Año'] <= 2016))    
# filters = (df['key']==key)

tmp = df_final[filters]

display(tmp.head())
fig = px.line(tmp, x='Fecha', y='Valor',title=f"Serie de tiempo para: ", 
              hover_data={'Semana': True, 'Año': True},
              color = 'Ax_003', markers = True)
fig.update_layout(height=800)
fig.show()

,Fecha,Cuadro,Estado,Codigo Padecimiento,Ax_003,Año,Semana,Padecimiento_ref,Ax_001,Ax_002,Valor,Padecimiento
60837,2014-01-06,CUADRO 17.,Chihuahua,F32,Acum.,2014,2,NaN,NaN,NaN,NaN,DepresiónCIE-10ª REV.F32
60838,2014-01-13,CUADRO 17.,Chihuahua,F32,Acum.,2014,3,NaN,NaN,NaN,NaN,DepresiónCIE-10ª REV.F32
60839,2014-01-20,CUADRO 17.,Chihuahua,F32,Acum.,2014,4,NaN,NaN,NaN,NaN,DepresiónCIE-10ª REV.F32
60840,2014-01-27,CUADRO 17.,Chihuahua,F32,Acum.,2014,5,NaN,NaN,NaN,NaN,DepresiónCIE-10ª REV.F32
60841,2014-02-03,CUADRO 17.,Chihuahua,F32,Acum.,2014,6,NaN,NaN,NaN,NaN,DepresiónCIE-10ª REV.F32


# 🔸 Combinación ('CUADRO 17.3', 'Zacatecas', 'Z91.5', 'H')

In [27]:
title = "Combinación ('CUADRO 17.3', 'Zacatecas', 'Z91.5', 'M')"

# # key = 'CUADRO 17.|Aguascalientes|G20|Acum.'
# key = 'CUADRO 17.|Michoacán|G20|Acum.'
filters = (df['Cuadro'] == 'CUADRO 17.3') \
          & (df['Estado']== 'Zacatecas') \
          & (df['Codigo Padecimiento'] == 'Z91.5') \
          & (df['Ax_003'] == 'M') 
        #   & ((df['Año'] >= 2014) & (df['Año'] <= 2016))    
# filters = (df['key']==key)

tmp = df[filters]

display(tmp.head())
fig = px.line(tmp, x='Fecha', y='Valor',title=f"Serie de tiempo para: {title} (df)", 
              hover_data={'Semana': True, 'Año': True},
              color = 'Ax_003', markers = True)
fig.update_layout(height=800)
fig.show()

,Año,Semana,Cuadro,Estado,Padecimiento,Ax_001,Ax_002,Ax_003,Valor,Fecha,Codigo Padecimiento
302695,2023,2,CUADRO 17.3,Zacatecas,Historia Personal de Lesión AutoinfligidaInten...,2023,Acum.,M,0,2023-01-09,Z91.5
302959,2023,3,CUADRO 17.3,Zacatecas,Historia Personal de Lesión AutoinfligidaInten...,2023,Acum.,M,0,2023-01-16,Z91.5
303223,2023,4,CUADRO 17.3,Zacatecas,Historia Personal de Lesión AutoinfligidaInten...,2023,Acum.,M,0,2023-01-23,Z91.5
303487,2023,5,CUADRO 17.3,Zacatecas,Historia Personal de Lesión AutoinfligidaInten...,2023,Acum.,M,0,2023-01-30,Z91.5
303751,2023,6,CUADRO 17.3,Zacatecas,Historia Personal de Lesión AutoinfligidaInten...,2023,Acum.,M,1,2023-02-06,Z91.5


In [28]:

title = "Combinación ('CUADRO 17.3', 'Zacatecas', 'Z91.5', 'M')"
# # key = 'CUADRO 17.|Aguascalientes|G20|Acum.'
# key = 'CUADRO 17.|Michoacán|G20|Acum.'
filters = (df_final['Cuadro'] == 'CUADRO 17.3') \
          & (df_final['Estado']== 'Zacatecas') \
          & (df_final['Codigo Padecimiento'] == 'Z91.5') \
          & (df_final['Ax_003'] == 'M') 
        #   & ((df['Año'] >= 2014) & (df['Año'] <= 2016))    
# filters = (df['key']==key)

tmp = df_final[filters]

display(tmp.head())
fig = px.line(tmp, x='Fecha', y='Valor',title=f"Serie de tiempo para: {title} (df_final)", 
              hover_data={'Semana': True, 'Año': True},
              color = 'Ax_003', markers = True)
fig.update_layout(height=800)
fig.show()

,Fecha,Cuadro,Estado,Codigo Padecimiento,Ax_003,Año,Semana,Padecimiento_ref,Ax_001,Ax_002,Valor,Padecimiento
760095,2014-01-06,CUADRO 17.3,Zacatecas,Z91.5,M,2014,2,NaN,NaN,NaN,NaN,Historia Personal de Lesión AutoinfligidaInten...
760096,2014-01-13,CUADRO 17.3,Zacatecas,Z91.5,M,2014,3,NaN,NaN,NaN,NaN,Historia Personal de Lesión AutoinfligidaInten...
760097,2014-01-20,CUADRO 17.3,Zacatecas,Z91.5,M,2014,4,NaN,NaN,NaN,NaN,Historia Personal de Lesión AutoinfligidaInten...
760098,2014-01-27,CUADRO 17.3,Zacatecas,Z91.5,M,2014,5,NaN,NaN,NaN,NaN,Historia Personal de Lesión AutoinfligidaInten...
760099,2014-02-03,CUADRO 17.3,Zacatecas,Z91.5,M,2014,6,NaN,NaN,NaN,NaN,Historia Personal de Lesión AutoinfligidaInten...


# Sugerencia de Variable Exogena

In [ ]:
# Crea una columna llamada "Covid-19" que sea True si las fechas están entre el 30 de marzo de 2020 y el 9 de mayo de 2023, y False en caso contrario
df_final['Covid-19'] = df_final['Fecha'].between('2020-03-30', '2023-05-09').astype(int)